### install and load packages (RBERT and tensorflow)
* this takes a while the first time you run this 

In [ ]:
# devtools::install_github(
#   "jonathanbratt/RBERT", 
#   build_vignettes = TRUE
# )

# tensorflow::install_tensorflow(version = "1.13.1")

In [3]:
library(RBERT)
library(dplyr)

In [ ]:
# might need to re-install pillar if there's an error message 
# install.packages('pillar')

In [5]:
# load pre-trained BERT embeddings 
BERT_PRETRAINED_DIR <- RBERT::download_BERT_checkpoint(
  model = "bert_base_uncased"
)

### load data from okcupid 

In [ ]:
#load okcupid essays
df = read.csv('/srv/jeremy.yang.tmp/text/okcupid_text.csv')

In [75]:
dim(df)

[1] 59946   556

In [76]:
names(df)

[1] "age"                              "body_type"                       
  [3] "diet"                             "drinks"                          
  [5] "drugs"                            "education"                       
  [7] "essay0"                           "essay1"                          
  [9] "essay2"                           "essay3"                          
 [11] "essay4"                           "essay5"                          
 [13] "essay6"                           "essay7"                          
 [15] "essay8"                           "essay9"                          
 [17] "ethnicity"                        "height"                          
 [19] "income"                           "job"                             
 [21] "last_online"                      "location"                        
 [23] "offspring"                        "orientation"                     
 [25] "pets"                             "religion"                        
 [27] "sex"                              "sign"                            
 [29] "smokes"                           "speaks"                          
 [31] "status"                           "uuid"                            
 [33] "age_group"                        "body_fat"                        
 [35] "body_fit"                         "body_type_noanswer"              
 [37] "body_thin"                        "vegetarian"                      
 [39] "halal_kosher"                     "alcohol"                         
 [41] "drugs_rec"                        "education_rec"                   
 [43] "grad_school"                      "black"                           
 [45] "asian"                            "short"                           
 [47] "income_rec"                       "haskids"                         
 [49] "wantskids"                        "gay"                             
 [51] "dog_person"                       "religious"                       
 [53] "smoker"                           "taken"                           
 [55] "self_summary_essay0"              "self_summary_contains_url"       
 [57] "self_summary_count"               "self_summary_bing_negative"      
 [59] "self_summary_bing_positive"       "self_summary_bing_sentiment"     
 [61] "self_summary_afinn_sentiment"     "self_summary_anger"              
 [63] "self_summary_anticipation"        "self_summary_disgust"            
 [65] "self_summary_fear"                "self_summary_joy"                
 [67] "self_summary_negative"            "self_summary_positive"           
 [69] "self_summary_sadness"             "self_summary_surprise"           
 [71] "self_summary_trust"               "self_summary_emotion"            
 [73] "self_summary_Per_anger"           "self_summary_Per_anticipation"   
 [75] "self_summary_Per_disgust"         "self_summary_Per_fear"           
 [77] "self_summary_Per_joy"             "self_summary_Per_sadness"        
 [79] "self_summary_Per_surprise"        "self_summary_Per_trust"          
 [81] "life_current_essay1"              "life_current_contains_url"       
 [83] "life_current_count"               "life_current_bing_negative"      
 [85] "life_current_bing_positive"       "life_current_bing_sentiment"     
 [87] "life_current_afinn_sentiment"     "life_current_anger"              
 [89] "life_current_anticipation"        "life_current_disgust"            
 [91] "life_current_fear"                "life_current_joy"                
 [93] "life_current_negative"            "life_current_positive"           
 [95] "life_current_sadness"             "life_current_surprise"           
 [97] "life_current_trust"               "life_current_emotion"            
 [99] "life_current_Per_anger"           "life_current_Per_anticipation"   
[101] "life_current_Per_disgust"         "life_current_Per_fear"           
[103] "life_current_Per_joy"             "life_current_Per_sadness"        
[105] "life_current_Per_surprise"        "life_cur

### pre-process the essay text
* using essay 0 as an example 

In [77]:
essay0 = as.character(text$essay0)

In [82]:
length(essay0)

[1] 59946

In [78]:
essay0[1]

[1] "about me:<br />\n<br />\ni would love to think that i was some some kind of intellectual:\neither the dumbest smart guy, or the smartest dumb guy. can't say i\ncan tell the difference. i love to talk about ideas and concepts. i\nforge odd metaphors instead of reciting cliches. like the\nsimularities between a friend of mine's house and an underwater\nsalt mine. my favorite word is salt by the way (weird choice i\nknow). to me most things in life are better as metaphors. i seek to\nmake myself a little better everyday, in some productively lazy\nway. got tired of tying my shoes. considered hiring a five year\nold, but would probably have to tie both of our shoes... decided to\nonly wear leather shoes dress shoes.<br />\n<br />\nabout you:<br />\n<br />\nyou love to have really serious, really deep conversations about\nreally silly stuff. you have to be willing to snap me out of a\nlight hearted rant with a kiss. you don't have to be funny, but you\nhave to be able to make me laugh. you should be able to bend spoons\nwith your mind, and telepathically make me smile while i am still\nat work. you should love life, and be cool with just letting the\nwind blow. extra points for reading all this and guessing my\nfavorite video game (no hints given yet). and lastly you have a\ngood attention span."

In [79]:
# keep the punctuations as they might provide some contexual information 
test = gsub('<br />|\\n|','',essay0[1])

# alternatively: remove all punctuations and only keep numbers and letters  
# test = gsub("[^[:alnum:]]",'',essay0[1])

In [98]:
test

[1] "about me:i would love to think that i was some some kind of intellectual:either the dumbest smart guy, or the smartest dumb guy. can't say ican tell the difference. i love to talk about ideas and concepts. iforge odd metaphors instead of reciting cliches. like thesimularities between a friend of mine's house and an underwatersalt mine. my favorite word is salt by the way (weird choice iknow). to me most things in life are better as metaphors. i seek tomake myself a little better everyday, in some productively lazyway. got tired of tying my shoes. considered hiring a five yearold, but would probably have to tie both of our shoes... decided toonly wear leather shoes dress shoes.about you:you love to have really serious, really deep conversations aboutreally silly stuff. you have to be willing to snap me out of alight hearted rant with a kiss. you don't have to be funny, but youhave to be able to make me laugh. you should be able to bend spoonswith your mind, and telepathically make me smile while i am stillat work. you should love life, and be cool with just letting thewind blow. extra points for reading all this and guessing myfavorite video game (no hints given yet). and lastly you have agood attention span."

In [81]:
essay0 = gsub('<br />|\\n|','',essay0)

### extract embeddings from the final layer of hidden states
* for more details of BERT's architechure see [this](https://mccormickml.com/2019/05/14/BERT-word-embeddings-tutorial/) 

In [88]:
# use a small block of texts to check how long it takes
start_time <- Sys.time()

BERT_feats <- extract_features(
  examples = essay0[1:100],
  ckpt_dir = BERT_PRETRAINED_DIR,
  layer_indexes = 12
)

end_time <- Sys.time()
end_time - start_time

Time difference of 1.12877 mins

In [91]:
# this is roughly how long it takes to process all the texts
(end_time - start_time)/100*length(essay0)

Time difference of 676.6525 mins

In [ ]:
# run it on the whole thing (max token length is currently set at 128, can change it if necessary)
BERT_feats <- extract_features(
  examples = essay0,
  ckpt_dir = BERT_PRETRAINED_DIR,
  layer_indexes = 12
)

In [92]:
# save the tokens and their corresponding features into a data frame
features = BERT_feats$output %>% select(-segment_index, -layer_index)

In [ ]:
head(features)

In [ ]:
# write the features into a csv
features = write.csv(features, 'bert_feats_essay0.csv')

### some utility functions that we are not using now but might be useful in the future

In [ ]:
# extract the final layer output vector for the "[CLS]" token of the first sentence. 
# output_vector1 <- BERT_feats$output %>%
#   dplyr::filter(
#     sequence_index == 1, 
#     token == "[CLS]", 
#     layer_index == 12
#   ) %>% 
#   dplyr::select(dplyr::starts_with("V")) %>% 
#   unlist()
# output_vector1

In [74]:
# Extract output vectors for all sentences...
# These vectors can be used as input features for downstream models.
# output_vectors <- BERT_feats$output %>% 
#   dplyr::filter(token_index == 1, layer_index == 12)
# output_vectors

In [56]:
# tokens <- tokenize_text(text = test,
#                         ckpt_dir = BERT_PRETRAINED_DIR)